In [5]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import sys
import datetime
import backtrader as bt
import backtrader.analyzers as btanalyzers

class CommInfoFractional(bt.CommissionInfo):
    def getsize(self, price, cash):
        '''Returns fractional size for cash operation @price'''
        return self.p.leverage * (cash / price)

### Buy and HODL
class BaselineStrategy(bt.Strategy):
    
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            cash = self.broker.get_cash()
            limit_price = self.data.close[0] * 1.01
            size = (cash * 0.95) / limit_price
            print(cash, self.data.close[0], limit_price, size)
            self.buy(exectype=bt.Order.Limit, size=size)

cerebro = bt.Cerebro()

from_date=datetime.datetime(2020, 1, 1)
to_date = datetime.datetime(2020, 1, 2)

# Create data feeds
data_ticks = bt.feeds.CCXT(exchange='coinbasepro', symbol='BTC/USD', name="btc_usd_tick",
                           timeframe=bt.TimeFrame.Minutes, compression=1, fromdate=from_date, todate=to_date)
cerebro.adddata(data_ticks)

cerebro.broker.setcash(10000.0)
cerebro.broker.addcommissioninfo(CommInfoFractional())
cerebro.broker.setcommission(commission=0.005, margin=False, mult=1.0, commtype=None, percabs=True, stocklike=False, interest=0.0, interest_long=False, leverage=1.0, automargin=False, name=None)

cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='annualreturn')
cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(btanalyzers.Returns, _name='returns')
cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='tradeanalyzer')

cerebro.addobserver(bt.observers.DrawDown)
# cerebro.addobserver(bt.observers.LogReturns)
# cerebro.addobserver(bt.observers.TimeReturn)

# Add the strategy
cerebro.addstrategy(BaselineStrategy)

# Run the strategy
strats = cerebro.run()

2020-01-01, Close, 7158.01
10000.0 7158.01 7229.5901 1.3140440700780531
2020-01-01, BUY EXECUTED, 7158.01
2020-01-01, Close, 7154.84
2020-01-01, Close, 7158.65
2020-01-01, Close, 7161.73
2020-01-01, Close, 7155.00
2020-01-01, Close, 7154.60
2020-01-01, Close, 7155.39
2020-01-01, Close, 7156.69
2020-01-01, Close, 7160.09
2020-01-01, Close, 7161.70
2020-01-01, Close, 7161.03
2020-01-01, Close, 7162.22
2020-01-01, Close, 7163.32
2020-01-01, Close, 7159.42
2020-01-01, Close, 7158.23
2020-01-01, Close, 7159.65
2020-01-01, Close, 7159.04
2020-01-01, Close, 7156.93
2020-01-01, Close, 7156.92
2020-01-01, Close, 7155.03
2020-01-01, Close, 7154.48
2020-01-01, Close, 7153.73
2020-01-01, Close, 7155.88
2020-01-01, Close, 7156.97
2020-01-01, Close, 7154.48
2020-01-01, Close, 7155.88
2020-01-01, Close, 7158.00
2020-01-01, Close, 7158.00
2020-01-01, Close, 7159.86
2020-01-01, Close, 7157.00
2020-01-01, Close, 7162.03
2020-01-01, Close, 7161.84
2020-01-01, Close, 7162.03
2020-01-01, Close, 7161.27
202

2020-01-01, Close, 7197.48
2020-01-01, Close, 7197.00
2020-01-01, Close, 7198.12
2020-01-01, Close, 7199.00
2020-01-01, Close, 7199.00
2020-01-01, Close, 7198.99
2020-01-01, Close, 7199.00
2020-01-01, Close, 7199.00
2020-01-01, Close, 7196.03
2020-01-01, Close, 7195.61
2020-01-01, Close, 7196.14
2020-01-01, Close, 7196.01
2020-01-01, Close, 7196.01
2020-01-01, Close, 7196.01
2020-01-01, Close, 7196.00
2020-01-01, Close, 7196.01
2020-01-01, Close, 7196.00
2020-01-01, Close, 7196.01
2020-01-01, Close, 7195.01
2020-01-01, Close, 7195.01
2020-01-01, Close, 7195.00
2020-01-01, Close, 7194.99
2020-01-01, Close, 7194.99
2020-01-01, Close, 7194.50
2020-01-01, Close, 7194.50
2020-01-01, Close, 7195.19
2020-01-01, Close, 7194.94
2020-01-01, Close, 7194.84
2020-01-01, Close, 7195.68
2020-01-01, Close, 7196.00
2020-01-01, Close, 7197.97
2020-01-01, Close, 7197.51
2020-01-01, Close, 7196.85
2020-01-01, Close, 7192.40
2020-01-01, Close, 7193.92
2020-01-01, Close, 7193.89
2020-01-01, Close, 7194.00
2

2020-01-01, Close, 7211.52
2020-01-01, Close, 7208.22
2020-01-01, Close, 7208.59
2020-01-01, Close, 7203.80
2020-01-01, Close, 7203.80
2020-01-01, Close, 7202.78
2020-01-01, Close, 7201.69
2020-01-01, Close, 7200.37
2020-01-01, Close, 7199.70
2020-01-01, Close, 7199.70
2020-01-01, Close, 7199.69
2020-01-01, Close, 7199.70
2020-01-01, Close, 7199.70
2020-01-01, Close, 7207.63
2020-01-01, Close, 7206.26
2020-01-01, Close, 7205.71
2020-01-01, Close, 7203.58
2020-01-01, Close, 7189.87
2020-01-01, Close, 7188.92
2020-01-01, Close, 7168.46
2020-01-01, Close, 7170.38
2020-01-01, Close, 7169.93
2020-01-01, Close, 7175.40
2020-01-01, Close, 7174.79
2020-01-01, Close, 7174.99
2020-01-01, Close, 7177.22
2020-01-01, Close, 7174.79
2020-01-01, Close, 7174.99
2020-01-01, Close, 7174.26
2020-01-01, Close, 7178.77
2020-01-01, Close, 7179.24
2020-01-01, Close, 7185.00
2020-01-01, Close, 7185.05
2020-01-01, Close, 7183.47
2020-01-01, Close, 7185.60
2020-01-01, Close, 7185.61
2020-01-01, Close, 7187.33
2

2020-01-01, Close, 7225.21
2020-01-01, Close, 7224.45
2020-01-01, Close, 7224.45
2020-01-01, Close, 7224.44
2020-01-01, Close, 7224.45
2020-01-01, Close, 7223.48
2020-01-01, Close, 7219.34
2020-01-01, Close, 7214.10
2020-01-01, Close, 7207.46
2020-01-01, Close, 7210.01
2020-01-01, Close, 7213.87
2020-01-01, Close, 7208.94
2020-01-01, Close, 7211.40
2020-01-01, Close, 7209.22
2020-01-01, Close, 7210.34
2020-01-01, Close, 7206.76
2020-01-01, Close, 7210.40
2020-01-01, Close, 7213.38
2020-01-01, Close, 7213.37
2020-01-01, Close, 7213.38
2020-01-01, Close, 7213.38
2020-01-01, Close, 7213.38
2020-01-01, Close, 7213.38
2020-01-01, Close, 7213.38
2020-01-01, Close, 7209.92
2020-01-01, Close, 7209.93
2020-01-01, Close, 7209.93
2020-01-01, Close, 7209.92
2020-01-01, Close, 7209.92
2020-01-01, Close, 7209.93
2020-01-01, Close, 7212.28
2020-01-01, Close, 7213.07
2020-01-01, Close, 7219.27
2020-01-01, Close, 7218.37
2020-01-01, Close, 7218.31
2020-01-01, Close, 7218.30
2020-01-01, Close, 7218.30
2

In [ ]:
print('Returns:', strats[0].analyzers.returns.pprint())
print('Sharpe Ratio:', strats[0].analyzers.mysharpe.pprint())
print('Annual Return:', strats[0].analyzers.annualreturn.pprint())
print('Drawdown:', strats[0].analyzers.drawdown.pprint())
print('Trade Analyzer:', strats[0].analyzers.tradeanalyzer.pprint())
cerebro.plot()